In [38]:
import requests
import json
import pandas as pd
import datetime
import numpy as np
import time
from bs4 import BeautifulSoup

In [2]:
with open('/Users/ChristopherKuzemka/Documents/GA/dsi_11/projects/capstone/env.json') as f:
    information = json.load(f)

Please note I am limited to 50 requests per minute.

1) Browse Quotes - Returns the cheapest quotes that meet your query. The prices come from our cached prices resulting from our users’ searches.

2) Browse Routes - Similar to Browse Quotes but with the quotes grouped by routes. This provides the cheapest destinations (countries, cities or airports) from our cached data.

3) Browse Dates - Similar to Browse Quotes but with the quotes grouped by outbound and inbound date. Useful to find the lowest price for a given route, over either a month or a 12 month period.

4) Browse Grid - Similar to Browse Dates but with some pre-processing on our side to output a two-dimensional array to easily display the response in calendar format.

In [40]:
current_flights = pd.read_csv('../data/current_flights.csv')
current_flights

,Unnamed: 0,faFlightID,ident,prefix,type,suffix,origin,destination,timeout,timestamp,...,lowLatitude,highLongitude,highLatitude,groundspeed,altitude,heading,altitudeStatus,updateType,altitudeChange,waypoints
0,0,DAL333-1590465975-fa-0008,DAL333,NaN,A321,NaN,KATL,KLAX,0,1590716390,...,32.94676,-84.44664,33.70005,448,300,264,NaN,A,D,33.64 -84.43 33.68 -84.28 33.81 -84.28 33.81 -...
1,1,KLM601-1590468354-airline-0005,KLM601,NaN,B77W,NaN,EHAM,KLAX,0,1590711509,...,33.95142,4.71741,66.06976,130,1,263,NaN,A,D,52.31 4.76 53.02 2.53 53.06 2.46 53.2 1.53 53....
2,2,VIR607-1590664542-ed-0002,VIR607,NaN,B789,NaN,EGLL,KLAX,0,1590711368,...,33.95091,-0.39345,66.09780,101,1,263,NaN,A,D,51.48 -0.46 51.56 -0.59 51.62 -0.68 51.7 -0.79...
3,3,DAL702-1590465982-fa-0006,DAL702,NaN,A321,NaN,KATL,KLAX,0,1590709847,...,33.64682,-84.44602,35.52932,117,1,263,NaN,A,D,33.64 -84.43 33.68 -84.28 33.79 -85 33.81 -85....
4,4,ACA572-1590468353-airline-0278,ACA572,NaN,A319,NaN,CYVR,KLAX,0,1590706589,...,33.95183,-118.17553,49.18984,109,1,263,NaN,A,D,49.19 -123.18 49.12 -123.26 49.12 -123.27 49.0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,100,EDV4708-1590468354-airline-0112,EDV4708,NaN,CRJ9,NaN,KAGS,KATL,0,1590664135,...,33.19607,-81.97964,33.63222,126,10,91,NaN,A,D,33.37 -81.96 33.39 -82.03 33.39 -82.04 33.38 -...
101,101,ABW341-1590583043-eb-0002,ABW341,NaN,B744,NaN,EBLG,KATL,0,1590631398,...,33.53333,5.56940,54.10380,132,9,90,,A,D,50.64 5.44 50.69 5.24 50.73 5.08 50.78 4.89 50...
102,102,DLH8176-1590641113-airline-0180,DLH8176,NaN,B772,NaN,EDDF,KATL,0,0,...,200.00000,-200.00000,-200.00000,0,0,0,NaN,NaN,NaN,NaN
103,103,CLX8626-1590641113-airline-0361,CLX8626,NaN,B744,NaN,ELLX,KATL,0,0,...,200.00000,-200.00000,-200.00000,0,0,0,NaN,NaN,NaN,NaN


In [47]:
flight_combinations = pd.read_csv('../data/flight_combinations.csv')
flight_combinations['origin'].unique()

array(['CYHM', 'CYUL', 'CYVR', 'CYYZ', 'EBBR', 'EBLG', 'EDDF', 'EDDN',
       'EDDP', 'EGLL', 'EHAM', 'EHBK', 'EIDW', 'ELLX', 'KABQ', 'KABY',
       'KAEX', 'KAFW', 'KAGS', 'KAPF', 'KAST', 'KATL', 'KAUS', 'KBFI',
       'KBIH', 'KBKV', 'KBOI', 'KBOS', 'KBUR', 'KBWI', 'KCVG', 'KCVO',
       'KDEN'], dtype=object)

In [44]:
flight_schedules = pd.read_csv('../data/flight_schedules.csv')
flight_schedules

,Unnamed: 0,ident,actual_ident,departuretime,arrival_time,origin,destination,aircrafttype,meal_service,seats_cabin_first,seats_cabin_business,seats_cabin_coach
0,0,UAL4282,ASQ4282,1588330800,1588340820,CYUL,KORD,E75L,Business: Refreshments / Economy: Food for sale,0,12,58
1,1,ACA7591,AC27591,1588335000,1588343880,CYUL,KORD,E75L,"Business: Breakfast / Economy: Breakfast, Food...",0,12,64
2,2,UAL8371,AC27591,1588335000,1588343880,CYUL,KORD,E75L,Business: Breakfast / Economy: Food for sale,0,12,64
3,3,UAL4245,ASQ4245,1588341060,1588351080,CYUL,KORD,E75L,Business: Refreshments / Economy: Food for sale,0,12,58
4,4,UAL8481,AC27595,1588353300,1588362000,CYUL,KORD,E75L,Business: Meal / Economy: Food for sale,0,12,64
...,...,...,...,...,...,...,...,...,...,...,...,...
5818,5818,UAL464,NaN,1590514200,1590523860,KDEN,KPDX,A319,Business: Snack or brunch / Economy: No meal,0,12,114
5819,5819,SWA378,NaN,1590520500,1590529500,KDEN,KPDX,B738,Economy: No meal,0,0,175
5820,5820,DLH9070,UAL393,1590539460,1590549120,KDEN,KPDX,A319,Business: Snack or brunch / Economy: No meal,0,12,114
5821,5821,UAL393,NaN,1590539460,1590549120,KDEN,KPDX,A319,Business: Snack or brunch / Economy: No meal,0,12,114


In [3]:
information.keys()

dict_keys(['FA_API_KEY', 'FA_USERNAME', 'x-rapidapi-host', 'x-rapidapi-key'])

In [5]:
host_name = information.get('x-rapidapi-host')
apiKey = information.get('x-rapidapi-key')

In [22]:
url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/autosuggest/v1.0/UK/GBP/en-GB/"

querystring = {"query":"Stockholm"}

headers = {
    'x-rapidapi-host': host_name,
    'x-rapidapi-key': apiKey
}

response = requests.get(url = url, headers=headers, params=querystring)

print(response.text)

{"Places":[{"PlaceId":"STOC-sky","PlaceName":"Stockholm","CountryId":"SE-sky","RegionId":"","CityId":"STOC-sky","CountryName":"Sweden"},{"PlaceId":"ARN-sky","PlaceName":"Stockholm Arlanda","CountryId":"SE-sky","RegionId":"","CityId":"STOC-sky","CountryName":"Sweden"},{"PlaceId":"NYO-sky","PlaceName":"Stockholm Skavsta","CountryId":"SE-sky","RegionId":"","CityId":"STOC-sky","CountryName":"Sweden"},{"PlaceId":"BMA-sky","PlaceName":"Stockholm Bromma","CountryId":"SE-sky","RegionId":"","CityId":"STOC-sky","CountryName":"Sweden"},{"PlaceId":"VST-sky","PlaceName":"Stockholm Vasteras","CountryId":"SE-sky","RegionId":"","CityId":"STOC-sky","CountryName":"Sweden"}]}


In [33]:
json_response = response.json()

In [34]:
json_response

{'Places': [{'PlaceId': 'STOC-sky',
   'PlaceName': 'Stockholm',
   'CountryId': 'SE-sky',
   'RegionId': '',
   'CityId': 'STOC-sky',
   'CountryName': 'Sweden'},
  {'PlaceId': 'ARN-sky',
   'PlaceName': 'Stockholm Arlanda',
   'CountryId': 'SE-sky',
   'RegionId': '',
   'CityId': 'STOC-sky',
   'CountryName': 'Sweden'},
  {'PlaceId': 'NYO-sky',
   'PlaceName': 'Stockholm Skavsta',
   'CountryId': 'SE-sky',
   'RegionId': '',
   'CityId': 'STOC-sky',
   'CountryName': 'Sweden'},
  {'PlaceId': 'BMA-sky',
   'PlaceName': 'Stockholm Bromma',
   'CountryId': 'SE-sky',
   'RegionId': '',
   'CityId': 'STOC-sky',
   'CountryName': 'Sweden'},
  {'PlaceId': 'VST-sky',
   'PlaceName': 'Stockholm Vasteras',
   'CountryId': 'SE-sky',
   'RegionId': '',
   'CityId': 'STOC-sky',
   'CountryName': 'Sweden'}]}

In [36]:
soup = BeautifulSoup(response.content)

In [37]:
soup

<html><body><p>{"Places":[{"PlaceId":"STOC-sky","PlaceName":"Stockholm","CountryId":"SE-sky","RegionId":"","CityId":"STOC-sky","CountryName":"Sweden"},{"PlaceId":"ARN-sky","PlaceName":"Stockholm Arlanda","CountryId":"SE-sky","RegionId":"","CityId":"STOC-sky","CountryName":"Sweden"},{"PlaceId":"NYO-sky","PlaceName":"Stockholm Skavsta","CountryId":"SE-sky","RegionId":"","CityId":"STOC-sky","CountryName":"Sweden"},{"PlaceId":"BMA-sky","PlaceName":"Stockholm Bromma","CountryId":"SE-sky","RegionId":"","CityId":"STOC-sky","CountryName":"Sweden"},{"PlaceId":"VST-sky","PlaceName":"Stockholm Vasteras","CountryId":"SE-sky","RegionId":"","CityId":"STOC-sky","CountryName":"Sweden"}]}</p></body></html>